In [ ]:
from dit import DiT, sample
import torch as t
import matplotlib.pyplot as plt

In [ ]:
model = DiT(28, 28, patch_size=2, n_classes=11, d=12*16, n_head=12, n_blocks=6)
model.load_state_dict(t.load("./checkpoints/model.pt"))

In [ ]:
model.eval()
model.cuda()

In [ ]:
z = t.randn(100, 1, 28, 28, dtype=model.dtype, device=model.device)
y = t.ones(100, dtype=t.long, device=model.device)
for num in range(11):
  print(num)
  if num > 0:
    cfg = 3
  else:
    cfg = 0
  x_pred = sample(model, z, y*0 + num, cfg=cfg, n_steps=30)

  fig, axes = plt.subplots(4, 8, figsize=(8, 4))
  axes = axes.flatten()

  for i in range(32):
      img = x_pred[i].squeeze().detach().cpu().clamp(-1, 1)
      img = (img + 1) / 2  # map back to [0,1]

      axes[i].imshow(img, cmap='gray')
      axes[i].axis('off')

  plt.tight_layout()
  plt.show()

